# **PRÀCTICA 1: Identificació d'idioma**

In [1]:
import re
import os
import math
import nltk
from nltk.collocations import TrigramCollocationFinder


In [2]:
texts_original = {}
directory = os.fsencode('./original_langId')
for file in os.listdir(directory):
	filename = os.fsdecode(file)
	directoryname = os.fsdecode(directory)
	with open(directoryname + '/' + filename, 'r', encoding="utf-8") as file:
		data = file.read()
		texts_original[filename] = data
	

In [3]:
def preprocess_text(text: str) -> str:
    new_text = ""
    sents = text.split('\n')
    for sent in sents:
        sent.strip()
        sent = sent.lower()
        sent = re.sub(r'\d', '', sent)
        sent = re.sub(r'\s+', ' ', sent)
        new_text += "  " + sent
    return new_text

In [4]:
texts_new = {}
for file, text in texts_original.items():
    texts_new[file] = preprocess_text(text)
    f = open("./new_langID/" + file, "w", encoding="utf-8")
    f.write(texts_new[file])
    f.close()

In [11]:
trigrams = {}
unique_chars = {}
for language in texts_new.keys():
	if 'tst' not in language:
		trigram_finder = TrigramCollocationFinder.from_words(texts_new[language])
		tmp = {key : value for key, value in trigram_finder.ngram_fd.items() if value > 5}
		
		trigrams[language] = tmp
		unique_chars[language] = len(set("".join("".join(a) for a in tmp.keys())))


In [13]:
def lidstone_smooth(language: str, trigram: tuple, lambda_value: float = 0.5):
    counts = trigrams[language]
    vocab = unique_chars[language] ** 3
    total = sum(counts.values())
    try:
        number = counts[trigram]
    except:
        number = 0
    probs = (number + lambda_value) / (total + lambda_value * vocab)
    return probs

def lidstone_total(text: str, language: str):
    trigram_finder = TrigramCollocationFinder.from_words(text)
    prob_sec = 0
    for trigram, num_instances in trigram_finder.ngram_fd.items():
        prob_sec += num_instances * math.log(lidstone_smooth(trigram=trigram, language=language))
    return prob_sec

text = "a pesar de la organización del movimiento, su paisano, abelande, critica la dispersión de las propuestas.   es decir, el bm entrega préstamos (evidentemente con elevados intereses que engrosan la deuda externa letal de los estados) a condición del cumplimiento de sus programas de ajuste estructural.   sin embargo, el juego no tenía construida una buena lan."
max_prob = ("None", float("-inf"))
for language in trigrams.keys():
    prob = lidstone_total(text, language)
    if prob > max_prob[1]:
        max_prob = (language, prob)
print(max_prob)

('nld_trn.txt', -699.8103507080053)


In [ ]:
import nltk
from nltk.util import ngrams
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import KneserNeyInterpolated

# Asegúrate de haber descargado los paquetes necesarios de NLTK la primera vez
# nltk.download('punkt')

# Ejemplo de texto para entrenar el modelo
text = "Este es un ejemplo de texto para demostrar cómo podemos aplicar suavizado de Kneser-Ney a trigramas. Este método ayuda a manejar n-gramas no observados."

# Tokenización del texto
tokens = nltk.word_tokenize(text, language='spanish')

# Preparación de los datos para el modelo de trigramas
train_data, padded_sents = padded_everygram_pipeline(3, [tokens])

# Creación del modelo de trigramas con Kneser-Ney Smoothing
model = KneserNeyInterpolated(3)  # El argumento es el orden del n-grama, 3 para trigramas

# Entrenamiento del modelo con los datos preparados
model.fit(train_data, padded_sents)

# Ejemplo de uso del modelo para generar la siguiente palabra después de un contexto dado
context = ['este', 'es']
print("Palabra(s) siguiente(s) más probable(s) después de 'este es':", model.generate(text_seed=context))

# Probabilidad de un trigram específico
trigram = ['para', 'demostrar', 'cómo']
print("Probabilidad del trigram 'para demostrar cómo':", model.score(trigram[-1], trigram[:-1]))
